## Libraries

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer, make_column_selector,ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler,RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate

## Load Data 

In [2]:
df_22 = pd.read_csv('../raw_data/fifa22/players_22.csv')
df_21 = pd.read_csv('../raw_data/fifa22/players_21.csv')
df_20 = pd.read_csv('../raw_data/fifa22/players_20.csv')
df_19 = pd.read_csv('../raw_data/fifa22/players_19.csv')
df_18 = pd.read_csv('../raw_data/fifa22/players_18.csv')
df_17 = pd.read_csv('../raw_data/fifa22/players_17.csv')
df_16 = pd.read_csv('../raw_data/fifa22/players_16.csv')
df_15 = pd.read_csv('../raw_data/fifa22/players_15.csv')

/var/folders/w7/n5m3c1_n3mn3b92rk5hw7msc0000gn/T/ipykernel_81768/3992483976.py:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_22 = pd.read_csv('../raw_data/fifa22/players_22.csv')
/var/folders/w7/n5m3c1_n3mn3b92rk5hw7msc0000gn/T/ipykernel_81768/3992483976.py:5: DtypeWarning: Columns (81,82,83,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv('../raw_data/fifa22/players_18.csv')
/var/folders/w7/n5m3c1_n3mn3b92rk5hw7msc0000gn/T/ipykernel_81768/3992483976.py:7: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  df_16 = pd.read_csv('../raw_data/fifa22/players_16.csv')
/var/folders/w7/n5m3c1_n3mn3b92rk5hw7msc0000gn/T/ipykernel_81768/3992483976.py:8: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  df_15 = pd.read_csv('../raw_data/fifa22/players_15.csv')


In [3]:
for index, col in enumerate(df_22.columns):
    print(f"{col:<26} | ", end="")
    if (index % 3 == 2):
        print("\n", end="")

sofifa_id                  | player_url                 | short_name                 | 
long_name                  | player_positions           | overall                    | 
potential                  | value_eur                  | wage_eur                   | 
age                        | dob                        | height_cm                  | 
weight_kg                  | club_team_id               | club_name                  | 
league_name                | league_level               | club_position              | 
club_jersey_number         | club_loaned_from           | club_joined                | 
club_contract_valid_until  | nationality_id             | nationality_name           | 
nation_team_id             | nation_position            | nation_jersey_number       | 
preferred_foot             | weak_foot                  | skill_moves                | 
international_reputation   | work_rate                  | body_type                  | 
real_face                  | rel

In [4]:
to_drop = ['club_logo_url','nation_flag_url','club_flag_url','nation_logo_url','player_face_url','dob','player_url',
           'real_face','nation_jersey_number','nation_position','club_loaned_from','long_name','player_url'
          ,'ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm',
           'rm','lwb','ldm', 'cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk','club_jersey_number','nationality_id','club_jersey_number']



In [5]:
df22 = df_22.drop(to_drop, axis = 1)
df21 = df_21.drop(to_drop, axis = 1)
df20 = df_20.drop(to_drop, axis = 1)
df19 = df_19.drop(to_drop, axis = 1)
df18 = df_18.drop(to_drop, axis = 1)
df17 = df_17.drop(to_drop, axis = 1)
df16 = df_16.drop(to_drop, axis = 1)
df15 = df_15.drop(to_drop, axis = 1)

In [6]:
def is_bench(d):
    if d in ("SUB", "RES", ""):
        return True 
    return False

In [7]:
dfs = [df18, df19, df20, df21,df22,df17,df16,df15]

for frame in dfs:
    frame['is_bench'] = frame['club_position'].apply(is_bench)
    frame['potential_diff'] = frame['potential'] - frame['overall']
    frame[['att_rate', 'def_rate']] = frame['work_rate'].str.split('/', 1, expand=True)

In [8]:
pos22 = df22['player_positions']
pos21 = df21['player_positions']
pos20 = df20['player_positions']
pos19 = df19['player_positions']
pos18 = df18['player_positions']
pos17 = df17['player_positions']
pos16 = df16['player_positions']
pos15 = df15['player_positions']


pref_pos_22 = []
pref_pos_21 = []
pref_pos_20 = []
pref_pos_19 = []
pref_pos_18 = []
pref_pos_17 = []
pref_pos_16 = []
pref_pos_15 = []

In [9]:
pos = [pos18, pos19, pos20, pos21,pos22,pos17,pos16,pos15]

for i in pos22:
    a = i.split(',')
    pref_pos_22.append(a[0])
df22['prefered_pos'] = pref_pos_22

for i in pos21:
    a = i.split(',')
    pref_pos_21.append(a[0])
df21['prefered_pos'] = pref_pos_21


for i in pos20:
    a = i.split(',')
    pref_pos_20.append(a[0])
df20['prefered_pos'] = pref_pos_20

for i in pos19:
    a = i.split(',')
    pref_pos_19.append(a[0])
df19['prefered_pos'] = pref_pos_19

for i in pos18:
    a = i.split(',')
    pref_pos_18.append(a[0])
df18['prefered_pos'] = pref_pos_18

for i in pos17:
    a = i.split(',')
    pref_pos_17.append(a[0])
df17['prefered_pos'] = pref_pos_17

for i in pos16:
    a = i.split(',')
    pref_pos_16.append(a[0])
df16['prefered_pos'] = pref_pos_16

for i in pos15:
    a = i.split(',')
    pref_pos_15.append(a[0])
df15['prefered_pos'] = pref_pos_15  



In [10]:
df22 = df22.drop('player_positions', axis = 1)
df21 = df21.drop('player_positions', axis = 1)
df20 = df20.drop('player_positions', axis = 1)
df19 = df19.drop('player_positions', axis = 1)
df18 = df18.drop('player_positions', axis = 1)
df17 = df17.drop('player_positions', axis = 1)
df16 = df16.drop('player_positions', axis = 1)
df15 = df15.drop('player_positions', axis = 1)

df22['prefered_pos'] = df22['prefered_pos'].astype('category')
df21['prefered_pos'] = df21['prefered_pos'].astype('category')
df20['prefered_pos'] = df20['prefered_pos'].astype('category')
df19['prefered_pos'] = df19['prefered_pos'].astype('category')
df18['prefered_pos'] = df18['prefered_pos'].astype('category')
df17['prefered_pos'] = df17['prefered_pos'].astype('category')
df16['prefered_pos'] = df16['prefered_pos'].astype('category')
df15['prefered_pos'] = df15['prefered_pos'].astype('category')




## Join Dataframes

In [11]:
df_all = df22.merge(df21, how='outer',left_on='sofifa_id', right_on='sofifa_id', suffixes=(None, '_21'))
df_all = df_all.merge(df20, how='outer',left_on='sofifa_id', right_on='sofifa_id', suffixes=(None,'_20'))
df_all = df_all.merge(df19, how='outer',left_on='sofifa_id', right_on='sofifa_id', suffixes=(None, '_19'))
df_all = df_all.merge(df18, how='outer',left_on='sofifa_id', right_on='sofifa_id', suffixes=(None, '_18'))
df_all = df_all.merge(df17, how='outer',left_on='sofifa_id', right_on='sofifa_id', suffixes=(None, '_17'))
df_all = df_all.merge(df16, how='outer',left_on='sofifa_id', right_on='sofifa_id', suffixes=(None, '_16'))
df_all = df_all.merge(df15, how='outer',left_on='sofifa_id', right_on='sofifa_id', suffixes=(None, '_15'))

In [12]:
for index, col in enumerate(df_all.columns):
    print(f"{col:<26} | ", end="")
    if (index % 3 == 2):
        print("\n", end="")

sofifa_id                  | short_name                 | overall                    | 
potential                  | value_eur                  | wage_eur                   | 
age                        | height_cm                  | weight_kg                  | 
club_team_id               | club_name                  | league_name                | 
league_level               | club_position              | club_joined                | 
club_contract_valid_until  | nationality_name           | nation_team_id             | 
preferred_foot             | weak_foot                  | skill_moves                | 
international_reputation   | work_rate                  | body_type                  | 
release_clause_eur         | player_tags                | player_traits              | 
pace                       | shooting                   | passing                    | 
dribbling                  | defending                  | physic                     | 
attacking_crossing         | att

In [13]:
df_all

,sofifa_id,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,...,goalkeeping_handling_15,goalkeeping_kicking_15,goalkeeping_positioning_15,goalkeeping_reflexes_15,goalkeeping_speed_15,is_bench_15,potential_diff_15,att_rate_15,def_rate_15,prefered_pos_15
0,158023,L. Messi,93.0,93.0,78000000.0,320000.0,34.0,170.0,72.0,73.0,...,11.0,15.0,14.0,8.0,NaN,False,2.0,Medium,Low,CF
1,188545,R. Lewandowski,92.0,92.0,119500000.0,270000.0,32.0,185.0,81.0,21.0,...,6.0,12.0,8.0,10.0,NaN,False,2.0,High,Medium,ST
2,20801,Cristiano Ronaldo,91.0,91.0,45000000.0,270000.0,36.0,187.0,83.0,11.0,...,11.0,15.0,14.0,11.0,NaN,False,0.0,High,Low,LW
3,190871,Neymar Jr,91.0,91.0,129000000.0,270000.0,29.0,175.0,68.0,73.0,...,9.0,15.0,15.0,11.0,NaN,False,5.0,High,Medium,LW
4,192985,K. De Bruyne,91.0,91.0,125500000.0,350000.0,30.0,181.0,70.0,10.0,...,13.0,5.0,10.0,13.0,NaN,False,5.0,High,Medium,CAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45624,225340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,6.0,8.0,12.0,NaN,True,6.0,Medium,Medium,CB
45625,221338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,10.0,12.0,5.0,NaN,True,12.0,Low,Medium,CB
45626,224829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,40.0,38.0,52.0,37.0,True,6.0,Medium,Medium,GK
45627,220549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,11.0,12.0,13.0,NaN,True,13.0,Medium,Medium,LM


In [14]:
df_plot=df_all[['sofifa_id','overall_15','overall_16','overall_17','overall_18','overall_19','overall_20','overall_21','overall']]


In [15]:
df_all

,sofifa_id,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,...,goalkeeping_handling_15,goalkeeping_kicking_15,goalkeeping_positioning_15,goalkeeping_reflexes_15,goalkeeping_speed_15,is_bench_15,potential_diff_15,att_rate_15,def_rate_15,prefered_pos_15
0,158023,L. Messi,93.0,93.0,78000000.0,320000.0,34.0,170.0,72.0,73.0,...,11.0,15.0,14.0,8.0,NaN,False,2.0,Medium,Low,CF
1,188545,R. Lewandowski,92.0,92.0,119500000.0,270000.0,32.0,185.0,81.0,21.0,...,6.0,12.0,8.0,10.0,NaN,False,2.0,High,Medium,ST
2,20801,Cristiano Ronaldo,91.0,91.0,45000000.0,270000.0,36.0,187.0,83.0,11.0,...,11.0,15.0,14.0,11.0,NaN,False,0.0,High,Low,LW
3,190871,Neymar Jr,91.0,91.0,129000000.0,270000.0,29.0,175.0,68.0,73.0,...,9.0,15.0,15.0,11.0,NaN,False,5.0,High,Medium,LW
4,192985,K. De Bruyne,91.0,91.0,125500000.0,350000.0,30.0,181.0,70.0,10.0,...,13.0,5.0,10.0,13.0,NaN,False,5.0,High,Medium,CAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45624,225340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,6.0,8.0,12.0,NaN,True,6.0,Medium,Medium,CB
45625,221338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,10.0,12.0,5.0,NaN,True,12.0,Low,Medium,CB
45626,224829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,40.0,38.0,52.0,37.0,True,6.0,Medium,Medium,GK
45627,220549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,11.0,12.0,13.0,NaN,True,13.0,Medium,Medium,LM


In [16]:
df_all['is_bench_21'] = df_all['is_bench_21'].astype('bool')
df_all['is_bench_20'] = df_all['is_bench_20'].astype('bool')
df_all['is_bench_19'] = df_all['is_bench_19'].astype('bool')
df_all['is_bench_18'] = df_all['is_bench_18'].astype('bool')
df_all['is_bench_17'] = df_all['is_bench_17'].astype('bool')
df_all['is_bench_16'] = df_all['is_bench_16'].astype('bool')
df_all['is_bench_15'] = df_all['is_bench_15'].astype('bool')

In [17]:
df_all.select_dtypes("bool").nunique()

is_bench_21    2
is_bench_20    2
is_bench_19    2
is_bench_18    2
is_bench_17    2
is_bench_16    2
is_bench_15    2
dtype: int64

In [18]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45629 entries, 0 to 45628
Columns: 577 entries, sofifa_id to prefered_pos_15
dtypes: bool(7), category(8), float64(456), int64(1), object(105)
memory usage: 196.7+ MB


In [19]:
# df_all.select_dtypes("float64").nunique()

df_all.work_rate.value_counts()

Medium/Medium    10015
High/Medium       3661
Medium/High       1880
High/High         1099
Medium/Low         810
High/Low           808
Low/Medium         466
Low/High           455
Low/Low             45
Name: work_rate, dtype: int64

In [20]:
feat_categorical_nunique = df_all.select_dtypes("object").nunique()
feat_categorical_nunique

short_name          18145
club_name             701
league_name            55
club_position          29
club_joined          1845
                    ...  
body_type_15           10
player_tags_15         59
player_traits_15      829
att_rate_15             3
def_rate_15             3
Length: 105, dtype: int64

In [21]:
feat_categorical_small = feat_categorical_nunique[feat_categorical_nunique < 10]
feat_categorical_small

preferred_foot       2
work_rate            9
is_bench             2
att_rate             3
def_rate             3
preferred_foot_21    2
work_rate_21         9
att_rate_21          3
def_rate_21          3
preferred_foot_20    2
work_rate_20         9
att_rate_20          3
def_rate_20          3
preferred_foot_19    2
work_rate_19         9
att_rate_19          3
def_rate_19          3
preferred_foot_18    2
work_rate_18         9
att_rate_18          3
def_rate_18          3
preferred_foot_17    2
work_rate_17         9
att_rate_17          3
def_rate_17          3
preferred_foot_16    2
work_rate_16         9
att_rate_16          3
def_rate_16          3
preferred_foot_15    2
work_rate_15         9
att_rate_15          3
def_rate_15          3
dtype: int64

In [22]:
wk_drop = ['work_rate','work_rate_21','work_rate_20','work_rate_19',
           'work_rate_18','work_rate_17','work_rate_16','work_rate_15']

df_all = df_all.drop(wk_drop, axis = 1)

In [23]:
df_all[['short_name','short_name_20','short_name_15','short_name_16','short_name_17']]

,short_name,short_name_20,short_name_15,short_name_16,short_name_17
0,L. Messi,L. Messi,L. Messi,L. Messi,L. Messi
1,R. Lewandowski,R. Lewandowski,R. Lewandowski,R. Lewandowski,R. Lewandowski
2,Cristiano Ronaldo,Cristiano Ronaldo,Cristiano Ronaldo,Cristiano Ronaldo,Cristiano Ronaldo
3,Neymar Jr,Neymar Jr,Neymar,Neymar,Neymar
4,K. De Bruyne,K. De Bruyne,K. De Bruyne,K. De Bruyne,K. De Bruyne
...,...,...,...,...,...
45624,NaN,NaN,B. Jackson,NaN,NaN
45625,NaN,NaN,M. Fissore,NaN,NaN
45626,NaN,NaN,H. Reynolds,NaN,NaN
45627,NaN,NaN,L. Davies,NaN,NaN


## Pipeline

In [24]:
# Preprocessor

num_transformer = make_pipeline(SimpleImputer(),MinMaxScaler())


cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)
cat_col = make_column_selector(dtype_include=['object','bool','category'])

preproc = make_column_transformer(
    (num_transformer, make_column_selector(dtype_include=['float64','int64'])),
    (cat_transformer, cat_col),
    remainder='passthrough')



# Add Estimator
pipe = make_pipeline(preproc,LinearRegression())
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x1501cef10>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x1501ceeb0>)])),
                ('linearregression', LinearRegression())])

In [25]:
X=df22.drop(columns='overall')
y=df22.overall

In [26]:
X_train22,X_test22,y_train22,y_test22=train_test_split(X,y,test_size=0.3)

In [ ]:
pipe.fit(X_train22,y_train22)

In [28]:
pipe.predict(X_test.iloc[0:2])

In [29]:
from sklearn.model_selection import cross_val_score,cross_validate

# pipe.fit(X,y)
# pipe.score(X,y)
# pipe.predict(new_data)
cv_results = cross_validate(pipe, X_train22, y_train22, cv=5, 
                            scoring=['r2']
                           )
pd.DataFrame(cv_results)

KeyboardInterrupt: 

In [ ]:
print('?')